In [1]:
!pip install ftfy

     |████████████████████████████████| 71kB 7.6MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=719b9385a479dc8ae0e439ed346986a0b281ab285cdd0f2fca058c2186b7a55b
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy


In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
import re #matching regular expression
import ftfy #fixes text for you:fixes all the junk characters
import nltk#natural language toolkit
import itertools #for iterations to matrix
import numpy as np #for array
import warnings #overcome warnings
warnings.filterwarnings("ignore")
import pandas as pd #read data
import pickle as pkl #store tokenizer
from pathlib import Path #create json file
from nltk import PorterStemmer #for stemming process
import matplotlib.pyplot as plt #plot confusion matrix
from nltk.corpus import stopwords #for remove stop words
from sklearn.model_selection import train_test_split #split data
from keras.models import Model, Sequential #adding layers one by one
from keras.optimizers import Adam #for optimize the training
from keras.preprocessing.text import Tokenizer #to genarate unique number for uniq words
from keras.preprocessing.sequence import pad_sequences #for making all the record has same shape
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score #for evaluate the model accuracy
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D #Layers for build model
from keras.layers import Bidirectional

In [7]:
np.random.seed(1234)

In [6]:
max_length=300
nb_max_words=100000
embedding_dim=10

In [8]:
fake_news = '/content/gdrive/MyDrive/Fake_News/Fake.csv'
true_news = '/content/gdrive/MyDrive/Fake_News/True.csv'

In [9]:
df_fake_news = pd.read_csv(fake_news)
df_true_news = pd.read_csv(true_news)

In [10]:
df_fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [11]:
df_true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [12]:
cList = pkl.load(open('/content/gdrive/MyDrive/Fake_News/cword_dict.pkl','rb'))

In [13]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [14]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [15]:
def clean_news(newses):
    cleaned_news = []
    for news in newses:
        news = str(news)
        if re.match("(\w+:\/\/\S+)", news) == None and len(news) > 5:
            news = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", news).split())
            news = ftfy.fix_text(news)
            news = expandContractions(news)
            news = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", news).split())
            stop_words = stopwords.words('english')
            word_tokens = nltk.word_tokenize(news) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            news = ' '.join(filtered_sentence)
            news = PorterStemmer().stem(news)
            cleaned_news.append(news)
    return cleaned_news

In [16]:
arr_fake_news = [x for x in df_fake_news['text']]
arr_true_news = [x for x in df_true_news['text']]

In [17]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
cleaned_fake_news = clean_news(arr_fake_news)
cleaned_true_news = clean_news(arr_true_news)

In [19]:
# tokenizer = Tokenizer(num_words=nb_max_words)
# print(tokenizer)
# tokenizer.fit_on_texts(cleaned_fake_news + cleaned_true_news)
from keras.preprocessing.text import one_hot


In [20]:
# sequences_fake = tokenizer.texts_to_sequences(cleaned_fake_news)
# sequences_true = tokenizer.texts_to_sequences(cleaned_true_news)

sequences_fake=[one_hot(words,nb_max_words) for words in cleaned_fake_news]
sequences_true=[one_hot(words,nb_max_words) for words in cleaned_true_news]

In [21]:
data_f = pad_sequences(sequences_fake,padding='pre', maxlen=max_length)
data_t = pad_sequences(sequences_true,padding='pre', maxlen=max_length)
print('Shape of data_fake tensor:', data_f.shape)
print('Shape of data_true tensor:', data_t.shape)

Shape of data_fake tensor: (22755, 300)
Shape of data_true tensor: (21416, 300)


In [22]:
data = np.concatenate((data_f, data_t))

In [23]:
labels_f = np.ones(data_f.shape[0])
labels_t = np.zeros(data_t.shape[0])
print(labels_f.shape, labels_t.shape)

(22755,) (21416,)


In [24]:
labels = np.concatenate((labels_f, labels_t))

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.2, random_state=42)

In [27]:
model=Sequential()
model.add(Embedding(nb_max_words,embedding_dim,input_length=max_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 10)           1000000   
_________________________________________________________________
dropout (Dropout)            (None, 300, 10)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               88800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                12864     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [28]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
553/553 [==============================] - 308s 515ms/step - loss: 0.3095 - accuracy: 0.8344 - val_loss: 0.0521 - val_accuracy: 0.9851
Epoch 2/10
553/553 [==============================] - 283s 512ms/step - loss: 0.0380 - accuracy: 0.9885 - val_loss: 0.0315 - val_accuracy: 0.9899
Epoch 3/10
553/553 [==============================] - 286s 517ms/step - loss: 0.0323 - accuracy: 0.9923 - val_loss: 0.0572 - val_accuracy: 0.9821
Epoch 4/10
553/553 [==============================] - 286s 517ms/step - loss: 0.0230 - accuracy: 0.9936 - val_loss: 0.0346 - val_accuracy: 0.9921
Epoch 5/10
553/553 [==============================] - 285s 516ms/step - loss: 0.0136 - accuracy: 0.9962 - val_loss: 0.0384 - val_accuracy: 0.9887
Epoch 6/10
553/553 [==============================] - 284s 514ms/step - loss: 0.0121 - accuracy: 0.9962 - val_loss: 0.0471 - val_accuracy: 0.9917
Epoch 7/10
553/553 [==============================] - 286s 516ms/step - loss: 0.0046 - accuracy: 0.9988 - val_loss: 0.0483 -

In [29]:
model.save('BiLSTM_model (1).h5')